# A Knowledge Graph for the SEEKCommons Project

The Resource Hub is a *[knowledge graph](https://en.wikipedia.org/wiki/Knowledge_graph)* curated by the __[SEEKCommons project](https://seekcommons.org/)__ that connects the various resources and entities in the SEEKCommons project. The graph is built using the [Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page) database and is accessible via the [SPARQL](https://en.wikipedia.org/wiki/SPARQL)query language.



## Where to start?

The graph is centered around the concept of a *resource*. A resource can be anything from a researcher, a research organization, a software tool, a dataset or a publication. Each resource is connected to other resources through various relationships. For example, a publication can be connected to a researcher who created it, a project that funded it, a publication that used it, or a software tool that processed it.

### Researchers

In [ ]:
query = '''
SELECT ?researcher ?orcid ?name
WHERE
{
  # instance of (https://www.wikidata.org/wiki/Property:P31) = human (https://www.wikidata.org/wiki/Q5)
  ?researcher wdt:P31  wd:Q5 .
  # occupation (https://www.wikidata.org/wiki/Property:P106) = researcher (https://www.wikidata.org/wiki/Q1650915)
  ?researcher wdt:P106 wd:Q1650915 .
  # ORCID iD (https://www.wikidata.org/wiki/Property:P496) => ?orcid
  ?researcher wdt:P496 ?orcid .
  # name in native language (https://www.wikidata.org/wiki/Property:P1559) => ?name
  ?researcher wdt:P1559 ?name .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
LIMIT 1000
'''

In [ ]:
import python.data_extraction as DEX

data_extracter = DEX.WikiDataQueryResults(query)
df = data_extracter.load_as_dataframe()
df.head(10)